In [221]:
import pandas as pd
import random
import datetime as dt

In [241]:
mulai_pengajaran = dt.datetime(year=2020, month=9, day=21) #senin
df = pd.read_csv("tbl0.csv")
df.index.name = "index"

# Pendaftaran tutup setelah total jam 60|61|62

In [242]:
#pendaftaran ditutup saat total jam pengajar mencapai 60|61|62
#maka, sort database dengan waktu, ambil data dari awal hingga jam nya 60|61|62 sort
df = df.sort_values("waktu_dftr")
df = df.reset_index()
df["cumsum"] = df["jam"].cumsum()
limit_hours = [60, 61, 62]
from_limits = [df.where((df["cumsum"]==i)).dropna() for i in limit_hours]

print(df)


    index  Unnamed: 0     nama  waktu_dftr  jam  senin  selasa  rabu  kamis  \
0      49          49  right49  2020-09-12    2      1       1     1      0   
1      75          75  right75  2020-09-12    1      0       0     1      1   
2      80          80  right80  2020-09-12    2      1       0     0      0   
3      31          31  right31  2020-09-12    2      1       1     1      1   
4      32          32  right32  2020-09-12    3      1       1     0      1   
..    ...         ...      ...         ...  ...    ...     ...   ...    ...   
95     61          61  right61  2020-09-21    2      0       0     0      0   
96     76          76  right76  2020-09-21    2      1       1     0      1   
97     29          29  right29  2020-09-21    3      0       0     1      1   
98     42          42  right42  2020-09-21    1      1       0     1      1   
99     73          73  right73  2020-09-21    3      1       1     1      1   

    jumat  sabtu  cumsum  
0       1      0       2

In [243]:
print(df.iloc[21])

index                 57
Unnamed: 0            57
nama             right57
waktu_dftr    2020-09-13
jam                    3
senin                  1
selasa                 1
rabu                   1
kamis                  1
jumat                  0
sabtu                  1
cumsum                44
Name: 21, dtype: object


In [244]:
#hapus dari from_limits yang tidak ada datanya
from_limits = [i for i in from_limits if len(i["nama"])!=0]
print(from_limits)


[    index  Unnamed: 0     nama  waktu_dftr  jam  senin  selasa  rabu  kamis  \
29   21.0        21.0  right21  2020-09-14  3.0    0.0     0.0   0.0    0.0   

    jumat  sabtu  cumsum  
29    1.0    1.0    60.0  ]


In [245]:
#ambil yang terbanyak
data_limit = from_limits[-1]
print(data_limit)


    index  Unnamed: 0     nama  waktu_dftr  jam  senin  selasa  rabu  kamis  \
29   21.0        21.0  right21  2020-09-14  3.0    0.0     0.0   0.0    0.0   

    jumat  sabtu  cumsum  
29    1.0    1.0    60.0  


In [246]:
#dapatkan indexnya
index_limit = int(data_limit.index.tolist()[0])
print(index_limit)

29


In [247]:
#ambil data dengan slicing 
new_df = df.iloc[0:index_limit+1]
del new_df["index"]
del new_df["Unnamed: 0"]
print(new_df)

new_df.to_csv("temp0.csv")

       nama  waktu_dftr  jam  senin  selasa  rabu  kamis  jumat  sabtu  cumsum
0   right49  2020-09-12    2      1       1     1      0      1      0       2
1   right75  2020-09-12    1      0       0     1      1      1      1       3
2   right80  2020-09-12    2      1       0     0      0      0      0       5
3   right31  2020-09-12    2      1       1     1      1      1      0       7
4   right32  2020-09-12    3      1       1     0      1      0      1      10
5   right83  2020-09-12    3      0       0     0      0      0      1      13
6   right87  2020-09-12    2      0       0     1      0      0      1      15
7   right33  2020-09-12    3      1       0     0      1      0      1      18
8   right34  2020-09-12    2      1       1     1      1      1      1      20
9   right14  2020-09-12    2      1       0     0      1      1      1      22
10  right59  2020-09-12    1      1       1     0      1      1      0      23
11  right13  2020-09-12    1      0       1     0   

# Sedapat mungkin 48 jam

In [249]:
all_days = "senin selasa rabu kamis jumat sabtu".split(" ")

df1 = new_df.copy()

#tambahkan kolom jumlah hari mengajar
df1["day_counts"] = df1[all_days].sum(axis=1)
# new_df = new_df.sort_values("day_counts")
# new_df.reset_index(inplace=True)
# del new_df["index"]
# del new_df["level_0"]

#tambahkan kolom total_hours
df1["total_hours"] = df1["day_counts"]*df1["jam"]
print(df1)

       nama  waktu_dftr  jam  senin  selasa  rabu  kamis  jumat  sabtu  \
0   right49  2020-09-12    2      1       1     1      0      1      0   
1   right75  2020-09-12    1      0       0     1      1      1      1   
2   right80  2020-09-12    2      1       0     0      0      0      0   
3   right31  2020-09-12    2      1       1     1      1      1      0   
4   right32  2020-09-12    3      1       1     0      1      0      1   
5   right83  2020-09-12    3      0       0     0      0      0      1   
6   right87  2020-09-12    2      0       0     1      0      0      1   
7   right33  2020-09-12    3      1       0     0      1      0      1   
8   right34  2020-09-12    2      1       1     1      1      1      1   
9   right14  2020-09-12    2      1       0     0      1      1      1   
10  right59  2020-09-12    1      1       1     0      1      1      0   
11  right13  2020-09-12    1      0       1     0      1      0      1   
12  right93  2020-09-12    2      1   

In [260]:
#dari day_counts terbanyak, ambil yang total_hours paling dikit
day_counts_unique = sorted(df1["day_counts"].unique())
print(day_counts_unique)

for dc in day_counts_unique:
    df_temp0 = df1[df1["day_counts"]==dc]
    print(df_temp0)
    

# total_hours_unique = sorted(df1["total_hours"].unique())
# print(total_hours_unique)






[1, 2, 3, 4, 5, 6]
       nama  waktu_dftr  jam  senin  selasa  rabu  kamis  jumat  sabtu  \
2   right80  2020-09-12    2      1       0     0      0      0      0   
5   right83  2020-09-12    3      0       0     0      0      0      1   
13  right58  2020-09-12    2      0       0     1      0      0      0   
14  right39  2020-09-12    1      0       0     0      0      1      0   
20  right36  2020-09-13    2      1       0     0      0      0      0   
25  right11  2020-09-13    1      0       0     1      0      0      0   
28   right6  2020-09-14    3      0       0     0      0      1      0   

    cumsum  day_counts  total_hours  
2        5           1            2  
5       13           1            3  
13      28           1            2  
14      29           1            1  
20      41           1            2  
25      52           1            1  
28      57           1            3  
       nama  waktu_dftr  jam  senin  selasa  rabu  kamis  jumat  sabtu  \
6   right8

In [315]:
df2 = df1.sort_values(["total_hours", "day_counts"])
df2.reset_index(inplace=True)
del df2["cumsum"]
del df2["index"]
df2


,nama,waktu_dftr,jam,senin,selasa,rabu,kamis,jumat,sabtu,day_counts,total_hours
0,right39,2020-09-12,1,0,0,0,0,1,0,1,1
1,right11,2020-09-13,1,0,0,1,0,0,0,1,1
2,right80,2020-09-12,2,1,0,0,0,0,0,1,2
3,right58,2020-09-12,2,0,0,1,0,0,0,1,2
4,right36,2020-09-13,2,1,0,0,0,0,0,1,2
5,right66,2020-09-13,1,0,1,0,0,1,0,2,2
6,right83,2020-09-12,3,0,0,0,0,0,1,1,3
7,right6,2020-09-14,3,0,0,0,0,1,0,1,3
8,right13,2020-09-12,1,0,1,0,1,0,1,3,3
9,right87,2020-09-12,2,0,0,1,0,0,1,2,4


In [331]:
#peminat hari2
dict_peminat_hari = dict()
sum_peminat_hari = [sum(df2[i]) for i in all_days]

index = 0
for day in all_days:
    dict_peminat_hari[day] = sum_peminat_hari[index]
    index+=1

sorted_hari_peminat = sorted(dict_peminat_hari, key=dict_peminat_hari.get)
# print(sorted_hari_peminat)

hari_nama = dict()

for hari in sorted_hari_peminat:
    yg_bs_hari_x = df2[df2[hari]==1]
    yg_bs_hari_x["cumsum_hari_x"] = (yg_bs_hari_x[hari]*yg_bs_hari_x["jam"]).cumsum()
    approach_8 = yg_bs_hari_x.where((yg_bs_hari_x["cumsum_hari_x"]<=8)).dropna()
#     print(approach_8)
    hari_nama[hari] = [list(approach_8["nama"]), list(approach_8["jam"])]
    
print(hari_nama)

{'rabu': [['right11', 'right58', 'right87', 'right75', 'right50'], [1.0, 2.0, 2.0, 1.0, 2.0]], 'jumat': [['right39', 'right66', 'right6', 'right75', 'right59'], [1.0, 1.0, 3.0, 1.0, 1.0]], 'selasa': [['right66', 'right13', 'right59', 'right90', 'right94', 'right46', 'right49'], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0]], 'senin': [['right80', 'right36', 'right59', 'right90', 'right94', 'right46'], [2.0, 2.0, 1.0, 1.0, 1.0, 1.0]], 'kamis': [['right13', 'right75', 'right59', 'right50', 'right90', 'right94', 'right46'], [1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0]], 'sabtu': [['right83', 'right13', 'right87', 'right75'], [3.0, 1.0, 2.0, 1.0]]}


/home/kevin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [333]:
#konfirmasi hasil -> harusnya tiap hari ada approach 8
dict_jamperhari = dict()
for hari in hari_nama:
    jam = 0
    nama_list = hari_nama[hari][0]
    for nama in nama_list:
        jam += (df2[df2["nama"]==nama]["jam"] * df2[df2["nama"]==nama][hari]).tolist()[0]
    dict_jamperhari[hari] = jam
    
print(dict_jamperhari)

{'rabu': 8, 'jumat': 7, 'selasa': 8, 'senin': 8, 'kamis': 8, 'sabtu': 7}
